In [ ]:
!pip install gradio requests


In [1]:
import os

HF_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
HF_MODEL = "meta-llama/Llama-3.1-8B-Instruct"

In [ ]:
import requests
import json

HF_MODEL = "meta-llama/Llama-3.1-8B-Instruct"
HF_API_KEY = "HUGGINGFACE_API_KEY"

def hf_generate_itinerary(prompt, max_tokens=3200, temperature=0.7):
    url = "https://router.huggingface.co/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {HF_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": HF_MODEL,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "max_tokens": max_tokens,
        "temperature": temperature
    }

    resp = requests.post(url, headers=headers, json=payload)
    print("Status code:", resp.status_code)

    try:
        data = resp.json()
    except json.JSONDecodeError:
        return f"⚠️ Non-JSON response:\n{resp.text}"

    if resp.status_code == 200:
        return data["choices"][0]["message"]["content"]
    else:
        return f"⚠️ Error {resp.status_code}: {data}"

In [ ]:
print(hf_generate_itinerary("Write Day 1, Day 2, Day 3,day 4, day 5 plan for Jaipur"))


Status code: 200
Jaipur, the vibrant capital city of Rajasthan, India, is a treasure trove of history, architecture, and culture. Here's a 5-day itinerary for exploring the best of Jaipur:

**Day 1: Arrival and Exploration of the City**

* 9:00 AM - Arrive in Jaipur and check-in to your hotel.
* 10:00 AM - Start the day with a visit to the City Palace, a majestic complex that showcases the rich history and architecture of Jaipur. Explore the intricate stone carvings, ornate doors, and beautifully crafted paintings.
* 1:00 PM - Take a break for lunch at a local restaurant, trying some of the city's famous cuisine like Dal Baati Churma.
* 2:30 PM - Visit the adjacent Jantar Mantar, an ancient astronomical observatory that features an array of stunning architectural structures, including the world's largest stone sundial.
* 5:00 PM - Explore the bustling streets of the old city, taking in the sights and sounds of the local markets and shops.
* 7:00 PM - Enjoy a traditional Rajasthani dinn

In [ ]:
print(
    hf_generate_itinerary(
        "Give me one fun fact about Jaipur, India."
    )
)

Status code: 200
One fun fact about Jaipur, India is that it is home to the famous Amer Fort, which has a unique reflection pool. During sunrise and sunset, the fort's reflection creates a breathtaking and beautiful illusion, often referred to as the 'Miracle Pool'. This is due to the fort being made of red sandstone and the surrounding pool being filled with water, creating a stunning visual effect.


In [ ]:
def google_maps_link(place_name, city_or_country=None):
    place_name = place_name.strip()
    place_name = place_name.replace("\n", " ")

    query = place_name
    if city_or_country:
        query += f", {city_or_country}"

    base = "https://www.google.com/maps/search/?api=1&query="
    return base + requests.utils.quote(query)

In [ ]:
import re
def extract_places(text):
    places = set()
    lines = text.split("\n")

    keywords = ["visit", "explore", "see", "go to", "tour", "stop at", "discover"]

    for line in lines:
        line_clean = line.strip()

        for key in keywords:
            if key in line_clean.lower():
                # Remove bullet points and emojis
                line_clean = re.sub(r"[^\w\s]", "", line_clean)
                words = line_clean.split()

                # Take last 2–4 words as place name
                if len(words) >= 2:
                    place = " ".join(words[-4:])
                    places.add(place)

    return list(places)


In [ ]:
from datetime import datetime

def build_prompt(destination, budget, start_date, end_date, interests):
    try:
        d1 = datetime.strptime(start_date, "%Y-%m-%d")
        d2 = datetime.strptime(end_date, "%Y-%m-%d")
        days = (d2 - d1).days + 1
    except:
        days = "a few"

    interests_text = interests if interests else "student-friendly experiences"

    return f"""
You are an expert student travel planner.

Create a personalized, budget-friendly itinerary for {destination}.

Constraints:
- Budget: INR {budget}
- Trip duration: {days} days
- Travel dates: from {start_date} to {end_date}
- Interests: {interests_text}

IMPORTANT INSTRUCTIONS (MUST FOLLOW):
- You MUST create a separate section for EACH day.
- Label sections clearly as: Day 1, Day 2, Day 3, etc.
- Do NOT stop after Day 1.
- Cover ALL {days} days completely.

For EACH day, include:
- Morning, Afternoon, and Evening plan
- Estimated daily cost (INR)
- Local transport tips
- Safety notes
- Free or low-cost options

Additionally:
- List 3–5 must-see places with short descriptions
- Include Google Maps links for these places

Use a clear, friendly, student-focused tone.
Use bullet points and clean formatting.
If the response is long, continue until ALL days are fully covered.
"""


In [ ]:
def plan_trip(destination, budget, start_date, end_date, interests):
    yield "⏳ Please wait, generating your itinerary..."

    prompt = build_prompt(destination, budget, start_date, end_date, interests)
    itinerary = hf_generate_itinerary(prompt)

    yield itinerary

In [ ]:
print(plan_trip("Delhi", "15000", "2026-01-16", "2026-01-18", "history"))


Status code: 200
**Delhi: A Historical Adventure Awaits!**

**Must-see Places:**

* **Red Fort** (Lal Qila): A UNESCO World Heritage Site and former Mughal palace. Explore the intricate architecture and learn about India's rich history. [Google Maps: https://goo.gl/maps/KUyT2o6kTaN3Z3UQ6](https://goo.gl/maps/KUyT2o6kTaN3Z3UQ6)
* **Qutub Minar**: The tallest minaret in India and another UNESCO World Heritage Site. Marvel at the intricate stone carvings and stunning architecture. [Google Maps: https://goo.gl/maps/DRkGZTkLbK7BnEjq7](https://goo.gl/maps/DRkGZTkLbK7BnEjq7)
* **India Gate**: A war memorial and iconic landmark in the heart of Delhi. Visit the surrounding gardens and admire the stunning architecture. [Google Maps: https://goo.gl/maps/8QgZp6iR6zB8sPbk9](https://goo.gl/maps/8QgZp6iR6zB8sPbk9)
* **Jama Masjid**: One of the largest mosques in India, built by the Mughal emperor Shah Jahan. Explore the stunning architecture and learn about the significance of the mosque. [Google Map

In [ ]:
import gradio as gr


def plan_trip(destination, budget, start_date, end_date, interests):
    # Show waiting message immediately
    yield "⏳ Please wait, generating your itinerary..."

    # Build prompt and generate itinerary
    prompt = build_prompt(destination, budget, start_date, end_date, interests)
    itinerary = hf_generate_itinerary(prompt)

    # Extract places and add Google Maps links
    places = extract_places(itinerary)
    maps_links = "\n\n### Google Maps Links for Places:\n"
    for place in places:
        maps_links += f"- {place}: {google_maps_link(place, destination)}\n"


    yield itinerary + maps_links


with gr.Blocks(title="AI Travel Planner") as demo:
    gr.Markdown("## 🧳 AI Travel Planner for Students\nGet personalized itineraries powered by Llama 3.1.")

    with gr.Row():
        with gr.Column():
            destination = gr.Textbox(label="Destination", placeholder="e.g., Jaipur, India")
            budget = gr.Textbox(label="Budget (INR)", placeholder="15000")
            start_date = gr.Textbox(label="Start Date (YYYY-MM-DD)")
            end_date = gr.Textbox(label="End Date (YYYY-MM-DD)")
            interests = gr.Textbox(label="Interests", placeholder="history, street food, local markets")
            btn = gr.Button("Generate Itinerary")
        with gr.Column():
            output = gr.Markdown()

    # output (generator function)
    btn.click(plan_trip, inputs=[destination, budget, start_date, end_date, interests], outputs=output)

demo.launch()



It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b8c7b3ee1b25922454.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
